In [89]:
import pandas as pd
import sqlite3
from os import path
import json
from lxml import html
import requests
import os
import datetime
from IPython.display import clear_output

bdPath = path.join("C:/", "Users", "Giovani", "Documents", "GitHub","topicosIC", "Em Grupo", "Trabalho Final", "BD", "bd.db")
nicksPath = path.join("..", "baseDeNicks.csv")

jogadoresPath = path.join("jogadores.sql")
campeoesJogadorPath = path.join("campeoesJogador.sql")
partidasPath = path.join("partidas.sql")
jogadoresPartidaPath = path.join("jogadoresPartida.sql")
logPath = path.join("nicksAdicionados.log")

In [86]:
#bd = sqlite3.connect(bdPath) 

#Adiciona Elos
#elos = open('elos.sql', 'r').read()
#bd.executescript(elos)

#Adiciona Campeoes
#Criando inserts
#campeoesJson = json.load(open('lol-champions.json', 'rb'))
#n = len(campeoesJson)-1
#while (n>-1):
#    print("INSERT INTO campeao('campeaoId', 'nome') VALUES("+campeoesJson[n]['key']+", \""+campeoesJson[n]['name']+"\""+");") 
#    n=n-1
#campeoes = open('campeoes.sql', 'r').read()
#bd.executescript(campeoes)

#elo = pd.read_sql('SELECT * FROM elo', bd)
#campeoes = pd.read_sql('SELECT * FROM campeao', bd)

#bd.close()

In [60]:
#Obtem informações do jogador e gera inserts pro bd
def getJogadorData(nick):
    baseUrl = "http://br.op.gg"
    playerUrl = "/summoner/userName="

    urlMontada = baseUrl+playerUrl+nick
    request = requests.get(urlMontada)
    html2 = html.fromstring(request.content)
    getData = html2.xpath('//div[@class="GameListContainer"]')
    if getData==[]:
        return -1
    summonerId = getData[0].attrib['data-summoner-id']

    elo = str(html2.xpath('//div/span[@class="tierRank"]/text()')).replace('[', '').replace(']', '').replace('\'', '')
    if elo=="":
        elo = "Unranked"
    pdl = str(html2.xpath('//div/span[@class="LeaguePoints"]/text()')).replace('[', '').replace(']', '').replace('\\n', '').replace('\\t', '').replace('\'', '').replace(' LP', '').replace(',', '')
    vitorias = str(html2.xpath('//span/span[@class="wins"]/text()')).replace('[', '').replace(']', '').replace('W', '').replace('L', '').replace('\'', '').replace(' LP', '').replace(',', '')
    derrotas = str(html2.xpath('//span/span[@class="losses"]/text()')).replace('[', '').replace(']', '').replace('W', '').replace('L', '').replace('\'', '').replace(' LP', '').replace(',', '')
    winrate = str(html2.xpath('//span/span[@class="winratio"]/text()')).replace('[', '').replace(']', '').replace('%', '').replace('Win Ratio ', '').replace('\'', '').replace(' LP', '')
    
    #TODO: Corrigir pois posições são dinamicamente colocadas após carregamento da página
    #posicoes = str(html2.xpath('//ul[@class="Content"]/li/div[@class="PositionStatContent"]/div/text()')).replace('[', '').replace(']', '').replace('\'', '').replace(' ', '').split(',')
    #posicao1 = posicoes[0]
    #posicao2 = posicoes[1]
    posicao1 = ""
    posicao2 = ""
    
    #DEBUGMODE
    '''
    print("Nick: " + nick + 
          "\nID: " + summonerId + 
          "\nElo: " + elo + 
          " - " + pdl + " PDL" +
          "\nVitórias: " + vitorias + 
          "\nDerrotas: " + derrotas + 
          "\nTaxa de Vitória: " + winrate + "%" +
          "\nPosição 1: " + posicao1 + 
          "\nPosição 2: " + posicao2)
    '''
    
    jogadoresSql = open(jogadoresPath, "a")
    jogadoresSql.write("INSERT INTO jogador('summonerId', 'nick', 'elo', 'pdl', 'vitorias', 'derrotas', 'winrate', 'posicao1', 'posicao2') VALUES("+summonerId+","+"\'"+nick+"\'"+","+"\'"+elo+"\'"+","+pdl+","+vitorias+","+derrotas+","+winrate+","+"\'"+posicao1+"\'"+","+"\'"+posicao2+"\');\n")
    jogadoresSql.close()
    return summonerId 

In [46]:
#Obtem informações dos campeoes do jogador e gera inserts pro bd
def getCampeoesJogadorData(summonerId):
    baseUrl = "http://br.op.gg"
    champsUrl = "/summoner/champions/ajax/champions.rank/summonerId="
    season = "&season=12"

    urlMontada = baseUrl+champsUrl+summonerId+season
    request = requests.get(urlMontada)
    html2 = html.fromstring(request.content)

    nCampeoes = html2.xpath('//tr[@class="Row TopRanker"]')
    for i in range(len(nCampeoes)):
        pref = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="Rank Cell"]/text()')[i]
        nome = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="ChampionName Cell"]/a/text()')[i]
        vitorias = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="RatioGraph Cell"]/div/div/div[@class="Text Left"]/text()')    
        if i>=len(vitorias):
            vitorias = '0'
        else:
            vitorias = vitorias[i].replace("W", "")

        derrotas = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="RatioGraph Cell"]/div/div/div[@class="Text Right"]/text()')
        if i>=len(derrotas):
            derrotas = '0'
        else:
            derrotas = derrotas[i].replace("L", "")
        
        partidas = str(int(vitorias)+int(derrotas))
        winrate = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="RatioGraph Cell"]/div/span/text()')[i].replace("%", "")
        kda = html2.xpath('//tr[@class="Row TopRanker"]/td[contains(@class, \'KDA \')]')[i].attrib['data-value']

        #TODO: Corrigir separação de cada valor pois o site é mal estruturado nesse ponto
        #values = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="Value Cell"]/text()')[i].split(' ')
        #values = str(values).replace('\\n', "").replace('\\t', "").replace('[', "").replace(']', "").replace(',', "").replace('\'', "")
        #values = str(values).splitlines()
        #ouro = values[0]
        #cs = values[1]
        #danoCausado = values[4]
        #danoRecebido = values[5]

        #DEBUGMODE
        '''
        print("ID: " + summonerId + 
              " Campeão: " + nome + 
              " Preferência: " + pref + 
              " Partidas: " + partidas + 
              " Vitórias: " + vitorias + 
              " Derrotas: " + derrotas + 
              " Taxa de Vitória: " + winrate + "%" +
              " KDA: " + kda)
        '''

        #print("\nINSERT INTO campeao_jogador('summonerId', 'campeaoNome', 'campeaoPref', 'partidas', 'vitorias', 'derrotas', 'winrate', 'kda', 'ouro', 'cs', 'danoCausado', 'danoRecebido') VALUES("
             # +summonerId+","+"\'"+nome+"\'"+","+pref+","+vitorias+derrotas+","+vitorias+","+derrotas+","+winrate+","+","+kda+","+ouro+","+cs+","+danoCausado+","+danoRecebido+");")

        campeoesJogadorSql = open(campeoesJogadorPath, "a")
        campeoesJogadorSql.write("INSERT INTO campeao_jogador('summonerId', 'campeaoNome', 'campeaoPref', 'partidas', 'vitorias', 'derrotas', 'winrate', 'kda', 'ouro', 'cs', 'danoCausado', 'danoRecebido') VALUES("
          +summonerId+","+"\'"+nome+"\'"+","+pref+","+partidas+","+vitorias+","+derrotas+","+winrate+","+kda+","+'0'+","+'0'+","+'0'+","+'0'+");\n")
        campeoesJogadorSql.close()

In [47]:
from bs4 import BeautifulSoup
from selenium import webdriver
    
#Funcao de apoio - converte nick em id
def findIdByName(summonerNome):
    baseUrl = "http://br.op.gg"
    playerUrl = "/summoner/userName="   

    urlMontada = baseUrl+playerUrl+summonerNome
    request = requests.get(urlMontada)
    html2 = html.fromstring(request.content)
    getData = html2.xpath('//div[@class="GameListContainer"]')
    summonerId = getData[0].attrib['data-summoner-id']

    return summonerId
    
#Obtem informações das partidas do jogador e gera inserts pro bd
def getPartidasJogadorData(summonerId):   
    baseUrl = "http://br.op.gg"
    champsUrl = "/summoner/matches/ajax/averageAndList/startInfo=0&summonerId="
    partidaUrl = "/summoner/matches/ajax/detail/gameId="
    seasonType = "&type=soloranked&season=12"
    
    urlMontada = baseUrl+champsUrl+summonerId+seasonType
    driver = webdriver.Chrome()
    driver.get(urlMontada)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    listaPartidas = soup.findAll("div", {"class": "GameItemWrap"})
    
    import json
    try:
        js = json.loads(soup.text)
    except ValueError:
        return -1
    
    teste = BeautifulSoup(js["html"], 'html.parser')
    listaPartidas = teste.findAll("div", {"class": "GameItemWrap"})

    campeoes = teste.findAll("div", {"class": "ChampionName"})
    kdas = teste.findAll("span", {"class": "KDARatio"})
    kdas.pop(0)
    desc = teste.findAll("td", {"class": "SummonerName"})
    partidas = teste.findAll("div", {"class": "GameItem"})
    

    for i in range(len(listaPartidas)):
    #for i in range(1):
        sumId = partidas[i].get('data-summoner-id')
        gameTime = partidas[i].get('data-game-time')
        gameId = partidas[i].get('data-game-id')
        
        #Informações da partida
        urlMontada2 = baseUrl+partidaUrl+gameId+"&summonerId="+sumId+"&gameTime="+gameTime
        request = requests.get(urlMontada2)
        html2 = html.fromstring(request.text)
        
        values = []
        summary = html2.xpath('//div[contains(@class, \'Team\')]/div[@class="ObjectScore"]/text()') 
        for k in summary[1::2]:
            values.append(k.strip())            
        baroesBlue = values[0]
        dragoesBlue = values[1]
        torresBlue = values[2]
        baroesRed = values[3]
        dragoesRed = values[4]
        torresRed = values[5]
        
        dadosBlue = html2.xpath('//div[@class="text graph--data graph--data__left"]/text()')
        dadosRed = html2.xpath('//div[@class="text graph--data graph--data__right"]/text()')        
        killsBlue = dadosBlue[0]
        killsRed = dadosRed[0]
        ouroBlue = dadosBlue[1]
        ouroRed = dadosRed[1]
        
        gameResult = html2.xpath('//th[@class="HeaderCell"]/span[@class="GameResult"]/text()')
        if gameResult[0].replace("\t", "").replace("\n", "") == "Victory":
            if html2.xpath('//tr/th[@class="HeaderCell"]/text()')[1].replace("\t", "").replace("\n", "") == "(Blue Team)":
                vitoria = "0"
            else:
                vitoria = "1"
        if gameResult[0].replace("\t", "").replace("\n", "") == "Defeat":
            if html2.xpath('//tr/th[@class="HeaderCell"]/text()')[1].replace("\t", "").replace("\n", "") == "(Blue Team)":
                vitoria = "1"
            else:
                vitoria = "0"
            
        #DEBUGMODE PARTIDA  
        '''
        print("GameID: " + gameId  + 
              " Vitória: " + vitoria +              
              " BaroesBlue: " + baroesBlue +
              " DragoesBlue: " + dragoesBlue  + 
              " TorresBlue: " + torresBlue  + 
              " BaroesRed: " + baroesRed + 
              " DragoesRed: " + dragoesRed  + 
              " TorresRed: " + torresRed  + 
              " KillsBlue: " + killsBlue  + 
              " KillsRed: " + killsRed  + 
              " OuroBlue: " + ouroBlue  + 
              " OuroRed: " + ouroRed)
        '''

        #print("INSERT INTO partida('gameId', 'vitoria', 'baroesBlue', 'dragoesBlue', 'torresBlue', 'baroesRed', 'dragoesRed', 'torresRed', 'killsBlue', 'killsRed', 'ouroBlue', 'ouroRed') VALUES("
        #      +gameId+","+vitoria+","+baroesBlue+","+dragoesBlue+","+torresBlue+","+baroesRed+","+dragoesRed+","+torresRed+","+killsBlue+","+killsRed+","+ouroBlue+","+ouroRed+");\n")
        partidasSql = open(partidasPath, "a")
        partidasSql.write("INSERT INTO partida('gameId', 'vitoria', 'baroesBlue', 'dragoesBlue', 'torresBlue', 'baroesRed', 'dragoesRed', 'torresRed', 'killsBlue', 'killsRed', 'ouroBlue', 'ouroRed') VALUES("
              +gameId+","+vitoria+","+baroesBlue+","+dragoesBlue+","+torresBlue+","+baroesRed+","+dragoesRed+","+torresRed+","+killsBlue+","+killsRed+","+ouroBlue+","+ouroRed+");\n")
        partidasSql.close()

        
        
        #Jogadores da partida
        html3 = html.fromstring(request.text)
        nicks = html3.xpath('//td[@class="SummonerName Cell"]/a/text()') 
        campeoes = html2.xpath('//tr[contains(@class, \'Row\')]/td/a/div[contains(@class, \'tip\')]/text()')
        kdas = html2.xpath('//td[@class="KDA Cell"]/span[contains(@class, \'KDARatio\')]/text()') #.replace(":1", "")
        danos = html2.xpath('//div[@class="ChampionDamage"]/text()') #replace(",", "")
        wards = html2.xpath('//span[@class="SightWard"]/text()')
        css = html2.xpath('//div[@class="CS"]/text()')
        
        #DEBUGMODE JOGADORES DA PARTIDA  
        '''
        for i in range(10):
            print("GameID: " + gameId  + 
              " SummonerId: " + nicks[i] +              
              " Campeão: " + campeoes[i] +
              " KDA: " + kdas[i].replace(":1", "")  + 
              " Dano: " + danos[i].replace(",", "")  + 
              " Wards: " + wards[i] + 
              " CS: " + css[i])
        '''
        
        for i in range(10):
            summonerId = findIdByName(nicks[i])
            campeaoNome = campeoes[i]
            kdaPartida = kdas[i].replace(":1", "").replace("Perfect", "10.00")
            danoPartida = danos[i].replace(",", "")
            wardsPartida = wards[i]
            csPartida = css[i]
            #print("INSERT INTO jogador_partida('gameId', 'summonerId', 'campeaoNome', 'kdaPartida', 'danoPartida', 'wardsPartida', 'csPartida') VALUES("
            #  +gameId+","+summonerId+",\""+campeaoNome+"\","+kdaPartida+","+danoPartida+","+wardsPartida+","+csPartida+");\n")
            #'''
            jogadoresPartidaSql = open(jogadoresPartidaPath, "a")
            jogadoresPartidaSql.write("INSERT INTO jogador_partida('gameId', 'summonerId', 'campeaoNome', 'kdaPartida', 'danoPartida', 'wardsPartida', 'csPartida') VALUES("
              +gameId+","+summonerId+",\""+campeaoNome+"\","+kdaPartida+","+danoPartida+","+wardsPartida+","+csPartida+");\n")
            jogadoresPartidaSql.close()
            #'''

In [55]:
nick = pd.read_csv(nicksPath)
nicks = nick['nick']

In [83]:
for jogador in nicks:
    log = open(logPath, "a")
    logS = str(datetime.datetime.now())+" - "+jogador+"\n"
    print(logS, end='\r\r')
    log.write(logS)
    log.close()
    summonerId = getJogadorData(jogador)
    #if(summonerId!=-1):
    #    getCampeoesJogadorData(summonerId)
    #    getPartidasJogadorData(summonerId)

2019-01-20 21:27:28.000732 - I Love You All
2019-01-20 21:27:29.062433 -  virtuous boy
2019-01-20 21:27:30.243087 -  procuro duo fiel
2019-01-20 21:27:31.262529 -  FLAnálista
2019-01-20 21:27:32.581527 -  lucifer not luci
2019-01-20 21:27:33.994293 - Compre na Havan
2019-01-20 21:27:35.362204 -  selfish nights
2019-01-20 21:27:36.567146 -  embrace sadness
2019-01-20 21:27:37.720994 -  FLA Robo
2019-01-20 21:27:38.681708 -  CNB Hy0g4
2019-01-20 21:27:40.166460 -  fã do jao piola
2019-01-20 21:27:41.507518 -  CNB Aslan
2019-01-20 21:27:42.559826 -  Guspfc
2019-01-20 21:27:43.514262 -  woodboy123123
2019-01-20 21:27:44.701417 -  KBM titan1
2019-01-20 21:27:45.905049 -  esA
2019-01-20 21:27:47.141614 -  Klaus
2019-01-20 21:27:47.669921 -  Mandioquinhaa
2019-01-20 21:27:48.775269 -  Ávenger
2019-01-20 21:27:49.936642 -  cmb
2019-01-20 21:27:50.814483 -  RIDERSONTHESTORM
2019-01-20 21:27:52.077162 -  Jack BIack
2019-01-20 21:27:53.916705 -  anyyyy
2019-01-20 21:27:55.449870 -  Damage
2019-01

2019-01-20 21:31:17.862903 -  MecLordinho
2019-01-20 21:31:18.926419 -  9132674871683128
2019-01-20 21:31:20.197913 -  Keinny
2019-01-20 21:31:21.331299 -  thanks Grandpa
2019-01-20 21:31:22.584650 -  Clovao Bengala
2019-01-20 21:31:23.778581 -  Wartin
2019-01-20 21:31:24.707627 -  JG DIFF XD
2019-01-20 21:31:26.003207 -  caíozera
2019-01-20 21:31:27.223639 -  sayajireN
2019-01-20 21:31:28.423691 -  Amazing Zuao
2019-01-20 21:31:29.194881 -  PiuPiuPeiPeiPOW
2019-01-20 21:31:30.878638 -  Sarkis
2019-01-20 21:31:32.561988 -  Little Rookie1
2019-01-20 21:31:33.005459 -  Tetis
2019-01-20 21:31:34.048271 -  Sorä
2019-01-20 21:31:35.719377 -  ESEQ
2019-01-20 21:31:38.047642 -  detrevni
2019-01-20 21:31:39.069741 -  BOOOTA NO ALTO
2019-01-20 21:31:40.591542 -  Dimi
2019-01-20 21:31:41.595591 -  SMURF DO MUCA 1
2019-01-20 21:31:42.955748 -  Zude
2019-01-20 21:31:44.339962 -  Mind Reformed
2019-01-20 21:31:44.951355 -  Sneez1
2019-01-20 21:31:45.872334 -  Vento
2019-01-20 21:31:46.597642 -  üüü

2019-01-20 21:35:09.655105 -  cutting ties
2019-01-20 21:35:09.938097 -  Áckerman
2019-01-20 21:35:11.161686 -  Judeuz1nho
2019-01-20 21:35:11.648859 -  PIJACK
2019-01-20 21:35:12.380527 -  MeuPIPINo5euPOPO
2019-01-20 21:35:13.576358 -  RRRRRRRRRRRANK1
2019-01-20 21:35:14.054332 -  I Esquilo I
2019-01-20 21:35:15.016954 -  Mystr
2019-01-20 21:35:16.168959 -  Nayrü
2019-01-20 21:35:17.201997 -  n tenho ideia
2019-01-20 21:35:18.525124 -  lIlIlIlllllllIl
2019-01-20 21:35:19.043710 -  Goddess Banana
2019-01-20 21:35:20.172108 -  spider freire
2019-01-20 21:35:21.307125 -  ZöoD
2019-01-20 21:35:22.623685 -  Nether Grasp
2019-01-20 21:35:23.144760 -  Jukes
2019-01-20 21:35:24.175561 -  someone to lead
2019-01-20 21:35:25.350955 -  PEEL NA SORAKITA
2019-01-20 21:35:25.977426 -  MaikindredTV
2019-01-20 21:35:26.376547 -  Kina
2019-01-20 21:35:27.730185 -  CARLINHOS DIGDIN
2019-01-20 21:35:28.874650 -  Haechan
2019-01-20 21:35:29.593724 -  Vahvel
2019-01-20 21:35:30.420426 -  tavin atentado
20

2019-01-20 21:38:29.758087 -  Vlàdimir
2019-01-20 21:38:30.346342 -  Cimento VIII
2019-01-20 21:38:30.766881 -  g3x Krow1
2019-01-20 21:38:31.608968 -  lima aka boss
2019-01-20 21:38:32.664128 -  lilola
2019-01-20 21:38:33.545708 -  MEC A ZED
2019-01-20 21:38:33.937598 -  gebradetrans
2019-01-20 21:38:34.742501 -  tyty02
2019-01-20 21:38:35.951705 -  Fluxe
2019-01-20 21:38:37.294682 -  Kalias Aponte
2019-01-20 21:38:38.538736 -  Mano Morallius
2019-01-20 21:38:39.682887 -  SamRosa1
2019-01-20 21:38:40.743667 -  Yatsu
2019-01-20 21:38:41.834247 -  Like DusT
2019-01-20 21:38:42.791081 -  Pro Deita 
2019-01-20 21:38:43.898555 -  Xèm
2019-01-20 21:38:45.264208 -  wolfinho123
2019-01-20 21:38:46.508438 -  XaRiQ
2019-01-20 21:38:48.349448 -  Axe Azazel
2019-01-20 21:38:49.459962 -  Alfa Hyeong
2019-01-20 21:38:50.838557 -  Guston
2019-01-20 21:38:51.415629 -  Virte
2019-01-20 21:38:52.592479 -  Metal
2019-01-20 21:38:55.346579 -  FluffyyBolo
2019-01-20 21:38:56.051637 -  Blastbeat Lover
2019

2019-01-20 21:42:05.293874 -  reIaxa eu so bom
2019-01-20 21:42:06.748570 -   Toninho on fire
2019-01-20 21:42:07.200906 -  Cabriita
2019-01-20 21:42:07.795251 -  GzuisCriaEuMato
2019-01-20 21:42:09.034313 -  ncp
2019-01-20 21:42:09.512410 -  Top de CBLOL
2019-01-20 21:42:10.782606 -  Hollowjj
2019-01-20 21:42:11.236839 -  ZED CARISMÁTICO 
2019-01-20 21:42:12.520840 -  tuljeu0180612
2019-01-20 21:42:13.014717 -  pocasx
2019-01-20 21:42:14.406137 -  KRSTZADA MECTREF
2019-01-20 21:42:15.003132 -  7842681Y
2019-01-20 21:42:15.615530 -  CHU DEDO MOLE 
2019-01-20 21:42:16.056445 -  tata cazari
2019-01-20 21:42:16.744072 -  anjxs
2019-01-20 21:42:17.195911 -  ASDAFHDFGSDFS
2019-01-20 21:42:17.918663 -  Snippiest1
2019-01-20 21:42:19.249506 -  ME IS TANK SMOrc
2019-01-20 21:42:20.036377 -  Okhamy
2019-01-20 21:42:20.514151 -  RST Lothbrok
2019-01-20 21:42:21.388764 -  me big brain
2019-01-20 21:42:22.290601 -  Tsuda
2019-01-20 21:42:23.632026 -  18052899
2019-01-20 21:42:24.049652 -  FinalDea

...

2019-01-21 05:20:20.007916 -  quero ser papai
2019-01-21 05:20:21.255012 -  Potick
2019-01-21 05:20:22.679564 -  Give me a Loli
2019-01-21 05:20:24.840262 -  Pagodear
2019-01-21 05:20:26.669810 - N1aws
2019-01-21 05:20:27.838208 -  R       C      F
2019-01-21 05:20:29.243167 -  Matheus The God
2019-01-21 05:20:30.913024 -  twt sirius
2019-01-21 05:20:32.273735 -  pantamorte
2019-01-21 05:20:33.617280 -  1Zero1
2019-01-21 05:20:34.984514 -  john is a dog
2019-01-21 05:20:36.333427 -  HG DanTe RX
2019-01-21 05:20:37.774400 -  HHapy
2019-01-21 05:20:39.115309 -  Capitâo Tigre
2019-01-21 05:20:40.774765 -  Yasuo chapadasso
2019-01-21 05:20:42.057938 -  Singêd
2019-01-21 05:20:43.982099 -  CG Wolfz
2019-01-21 05:20:45.501708 -  BateBolo
2019-01-21 05:20:46.938337 -  Míaka
2019-01-21 05:20:48.665908 -  Não fica tisti 
2019-01-21 05:20:50.028333 -  MartialGodAsura
2019-01-21 05:20:51.350916 -  Markinho
2019-01-21 05:20:52.951134 -  TheColoussus
2019-01-21 05:20:54.076922 -  Saign
2019-01-21 0

2019-01-21 05:25:04.820502 -  CapetaNego
2019-01-21 05:25:06.474223 -  Dia das Bruxas
2019-01-21 05:25:08.741692 -  Gean Cesar
2019-01-21 05:25:10.373016 -  Marcossd2
2019-01-21 05:25:11.710261 -  Alexcbc
2019-01-21 05:25:12.419582 -  Masui
2019-01-21 05:25:13.933597 - Maxmorda
2019-01-21 05:25:15.280737 -  nico is a fish
2019-01-21 05:25:17.061542 -  how2 save a life
2019-01-21 05:25:18.834268 -  ROD OF MECHANICS
2019-01-21 05:25:20.276430 -  Ukiye
2019-01-21 05:25:21.676961 -  Rdgnolsd
2019-01-21 05:25:23.249781 -  LiIíth
2019-01-21 05:25:24.536064 -  quero me drogar
2019-01-21 05:25:25.002053 -  Dark3zin
2019-01-21 05:25:26.618867 -  ZéPikenin
2019-01-21 05:25:27.759460 -  Corno e Baiano
2019-01-21 05:25:29.268305 -  Gurlock
2019-01-21 05:25:31.355601 -  STW Juni0rbzl
2019-01-21 05:25:32.854403 -  EL D0NKEY K0NG
2019-01-21 05:25:33.571092 -  Houkaz In Peace
2019-01-21 05:25:35.163038 -  SouMonoShaco 
2019-01-21 05:25:36.499852 -  Smauel
2019-01-21 05:25:37.995954 -  Nink
2019-01-21 

2019-01-21 05:30:00.327475 -  Astora
2019-01-21 05:30:01.879566 -  IV ThuG
2019-01-21 05:30:03.476190 -  Vini Lima
2019-01-21 05:30:04.991837 -  BIG DUDU XD
2019-01-21 05:30:07.455212 -  Camillee
2019-01-21 05:30:10.045207 -  Braziliann2
2019-01-21 05:30:11.468875 - xHULKx
2019-01-21 05:30:12.682986 -  TRMBLN
2019-01-21 05:30:14.352574 -  LL Stylish Br 21
2019-01-21 05:30:17.400188 -  Pandle
2019-01-21 05:30:19.098312 -  Soyth
2019-01-21 05:30:20.723475 -  Teemo Cannabis
2019-01-21 05:30:21.854144 -  TiltedMan
2019-01-21 05:30:22.944071 -  Baeckl
2019-01-21 05:30:24.015271 -  I won 
2019-01-21 05:30:25.140934 -  teker
2019-01-21 05:30:26.389375 -  TOM4S TURB4NDO
2019-01-21 05:30:27.694695 -  juggernault
2019-01-21 05:30:28.406290 -  Im Minnie
2019-01-21 05:30:29.946513 -  Tryndacarry
2019-01-21 05:30:31.058632 -  PIOB borracha
2019-01-21 05:30:32.082190 -  desiderius
2019-01-21 05:30:33.637752 -  Astora
2019-01-21 05:30:35.225860 -  IV ThuG
2019-01-21 05:30:36.249136 -  Vini Lima
2019-

2019-01-21 05:34:48.661458 -  MatheusDatena
2019-01-21 05:34:50.170258 -  lil ney
2019-01-21 05:34:51.873054 -  Jogador Absoluto
2019-01-21 05:34:53.569429 -  TiozãoArlene
2019-01-21 05:34:55.483851 -  HallsB
2019-01-21 05:34:56.917746 -  Space Panda
2019-01-21 05:34:58.511792 -  mic9
2019-01-21 05:35:00.041619 -  ZEDE
2019-01-21 05:35:01.710323 - De Putrefação 
2019-01-21 05:35:03.411055 -  Uso calça Jinx
2019-01-21 05:35:04.867005 -  LaryBandida
2019-01-21 05:35:05.349266 -  levigc12
2019-01-21 05:35:06.079938 -  Kuchiki Rukyia
2019-01-21 05:35:06.556977 -  Hi im Gosu Dog 
2019-01-21 05:35:08.177534 -  SnowzHs
2019-01-21 05:35:09.599764 -  Ekko Is My Main
2019-01-21 05:35:11.100066 -  SÓ PERCO VELHO 
2019-01-21 05:35:11.850057 -  Reborn Source
2019-01-21 05:35:13.350011 -  munchkin
2019-01-21 05:35:14.677914 -  bob dos teclado1
2019-01-21 05:35:15.380212 -  Morett
2019-01-21 05:35:16.937049 -  Kadarin
2019-01-21 05:35:18.345206 -  Krelam
2019-01-21 05:35:19.642486 -  LOVEkillU
2019-0

ParserError: Document is empty

In [96]:
#Adiciona Jogadores
bd = sqlite3.connect(bdPath) 
jogadores = open(jogadoresPath, 'r').read()
bd.executescript(jogadores)
ids = pd.read_sql('SELECT (jogador.summonerId) FROM jogador', bd)
bd.close()

OperationalError: near ",": syntax error

In [82]:
#Apagar Arquivos de Teste
#if(os.path.isfile(logPath)):
#    os.remove(logPath)
#if(os.path.isfile(jogadoresPath)):
#    os.remove(jogadoresPath)
if(os.path.isfile(campeoesJogadorPath)):
    os.remove(campeoesJogadorPath)
if(os.path.isfile(partidasPath)):
    os.remove(partidasPath)
if(os.path.isfile(jogadoresPartidaPath)):
    os.remove(jogadoresPartidaPath)

In [ ]:
for idd in ids:
    getCampeoesJogadorData(idd)

In [94]:
ids

,summonerId
0,402032
1,404084
2,410074
3,410984
4,411669
5,418760
6,440670
7,441589
8,442237
9,447057
